In [ ]:
import pandas as pd

import networkx as nx
import pydot

import dowhy
from econml.dml import CausalForestDML
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
df = pd.read_csv('df_causal.csv') # Change file name and path accordingly

In [ ]:
# One potential DAG model for the transport system
causal_graph = """digraph {
    
    "Particle Density" -> "Hydraulic Conductivity";
    "Particle Density"-> Dispersivity;
    "Particle Density" -> "Adsorption Coefficient";
    "Particle Density" -> "Type-1 Sorption Fraction";  
    
    "Porosity" -> "Hydraulic Conductivity";
    "Porosity"-> Dispersivity;
    "Porosity" -> "Adsorption Coefficient";
    "Porosity" -> "Type-1 Sorption Fraction";         
    
    "Degree of Saturation" -> Dispersivity;   
    "Degree of Saturation" -> "Type-1 Sorption Fraction";    
    Distance -> Dispersivity;    
    Concentration -> "Type-2 Sorption Reaction Rate";
    
    "Ponded Water Depth" -> "Relative Velocity";    
    Flux -> "Relative Velocity";
    "Type-1 Sorption Fraction" -> "Relative Velocity";        
    "Degree of Saturation" -> "Relative Velocity";
    Distance ->  "Relative Velocity"; 
    Concentration -> "Relative Velocity";
    Horizontal -> "Relative Velocity";
    "Adsorption Coefficient" -> "Relative Velocity";        
    "Hydraulic Conductivity" -> "Relative Velocity";
    Dispersivity-> "Relative Velocity";
    "Type-2 Sorption Reaction Rate"-> "Relative Velocity";
    
}
"""

In [ ]:
pydot_graph = pydot.graph_from_dot_data(causal_graph.replace("\n", " "))[0]

In [ ]:
nx_graph = nx.drawing.nx_pydot.from_pydot(pydot_graph)

# Extract all edge pairs
edge_pairs = list(nx_graph.edges())

In [ ]:
df.describe()

In [ ]:
df = df.rename(columns={'Ponded Water': 'Ponded Water Depth'})

In [ ]:
def causal_model(treatment,outcome,graph,df):
    model = dowhy.CausalModel(data = df,
                        treatment = treatment,
                        outcome = outcome,
                        graph = graph)
    
    modifiers = model.get_effect_modifiers()
    confounders = model.get_common_causes()   

    estimand = model.identify_effect(proceed_when_unidentifiable=True)
    backdoor_var = estimand.backdoor_variables
    
    #  Linear
    estimate_li = model.estimate_effect(estimand,method_name = "backdoor.linear_regression", method_params = None, confidence_intervals = True)
    
    print(treatment,outcome,"############### Now refuting: Random Common Cause (Linear)#######################")
    res_random_li=model.refute_estimate(estimand,estimate_li, method_name="random_common_cause")    
    print(treatment,outcome,"############### Now refuting: Add Unobserved Common Cause (Linear)######################")
    res_unobserved_li=model.refute_estimate(estimand, estimate_li, method_name="add_unobserved_common_cause",
                                         confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                        effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)
    print(treatment,outcome,"############### Now refuting: Placebo (Linear)##############################")
    res_placebo_li=model.refute_estimate(estimand, estimate_li, method_name="placebo_treatment_refuter",placebo_type="permute")
    li_res = [estimate_li.value, estimate_li.get_confidence_intervals(),res_random_li,res_unobserved_li,res_placebo_li]

    #  DML
    if len(confounders)>0 or len(modifiers)>0:     
        
        est_nonparam = CausalForestDML(model_y=GradientBoostingRegressor(), model_t=GradientBoostingRegressor(),random_state=12)

        Y = df[outcome].values
        T = df[treatment].values

        args = [Y, T]


        if len(modifiers)== 0:
                        
            print('Special case: NO Effect Modifier!')
            kwargs = {'inference':'auto'}
            X = df[confounders].values
            kwargs['X'] = X
            W = None

            # Here in the special case, we use raw EconML interface instead of DoWhy wrapper or EconML wrapper in either package to avoid confusion
            est_nonparam.fit(*args, **kwargs)
            estimated_ate = te_pred.mean()
            te_pred = est_nonparam.effect(X)
            estimated_ate_ci = (est_nonparam.effect_interval(X)[0].mean(),est_nonparam.effect_interval(X)[1].mean())

            print(treatment,outcome,"############### NO DML REFUTATION!#######################")
            
            res_random_dml,res_unobserved_dml,res_placebo_dml = None,None,None   

            

        else:
            print('Ordinary case: has effect modifier and confounders.')
            
            estimate_dml = model.estimate_effect(estimand, method_name="backdoor.econml.dml.CausalForestDML",
                                 confidence_intervals=True,
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                              'model_t': GradientBoostingRegressor()},
                                               "fit_params":{}})      
            
            estimated_ate = estimate_dml.value
            te_pred = estimate_dml.cate_estimates
            estimated_ate_ci = estimate_dml.get_confidence_intervals()   

            print(treatment,outcome,"############### Now refuting: Random Common Cause (DML)##################")
            res_random_dml = model.refute_estimate(estimand, estimate_dml, method_name="random_common_cause")

            print(treatment,outcome,"############### Now refuting: Add Unobserved Common Cause (DML)##################")
            res_unobserved_dml =model.refute_estimate(estimand, estimate_dml, method_name="add_unobserved_common_cause",
                                                 confounders_effect_on_treatment="binary_flip", confounders_effect_on_outcome="linear",
                                                effect_strength_on_treatment=0.01, effect_strength_on_outcome=0.02)

            print(treatment,outcome,"############### Now refuting: Placebo (DML)##############################")
            res_placebo_dml = model.refute_estimate(estimand, estimate_dml, method_name="placebo_treatment_refuter", placebo_type="permute")
            


        dml_res = [estimated_ate, te_pred,estimated_ate_ci,res_random_dml,res_unobserved_dml,res_placebo_dml]        

    else:
        dml_res = None

    return li_res,dml_res,modifiers,confounders, backdoor_var

In [ ]:
results_full = []

In [ ]:
for pair in edge_pairs[:]:
    results_full.append(causal_model(*pair,causal_graph,df))

In [ ]:
for pair in edge_pairs[20:]:
    results_full.append(causal_model(*pair,causal_graph,df))

In [ ]:
df_res=pd.DataFrame(columns = 
                         ['treatment','outcome',
                          'ate_li','ci_li',
                          'rand_li','rand_li-p-val','rand_li-is_statistically_significant',
                          'unobserved_li','placebo_li','li-pl-p-val','li-pl_is_statistically_significant',
                          'ate_dml','ate2_dml','ci_dml',
                          'rand_dml','rand_dml-p-val','rand_dml-is_statistically_significant',
                          'unobserved_dml','placebo_dml','dml_pl_p_val','dml_pl_is_statistically_significant',
                          'modifiers','confounders','backdoor_var'])

In [ ]:
df_res['treatment'] = [ele[0] for ele in edge_pairs]
df_res['outcome'] = [ele[1] for ele in edge_pairs]

In [ ]:
df_res['ate_li'] = [x[0][0] for x in results_full]
df_res['ci_li'] = [x[0][1] for x in results_full]

In [ ]:
df_res['rand_li'] = [x[0][2].new_effect for x in results_full]
df_res['rand_li-p-val'] = [x[0][2].refutation_result['p_value'] for x in results_full]
df_res['rand_li-is_statistically_significant'] = [x[0][2].refutation_result['is_statistically_significant'] for x in results_full]
df_res['unobserved_li'] = [x[0][3].new_effect for x in results_full]
df_res['placebo_li'] = [x[0][4].new_effect for x in results_full]
df_res['li-pl-p-val'] = [x[0][4].refutation_result['p_value'] for x in results_full]
df_res['li-pl_is_statistically_significant'] = [x[0][4].refutation_result['is_statistically_significant'] for x in results_full]

In [ ]:
df_res['ate_dml'] = [x[1][0] if x[1] else None for x in results_full]
df_res['ate2_dml'] = [x[1][1] if x[1] else None for x in results_full]
df_res['ci_dml'] = [x[1][2] if x[1] else None for x in results_full]

In [ ]:
df_res['rand_dml'] = [x[1][3].new_effect if x[1] else None for x in results_full]
df_res['rand_dml-p-val'] = [x[1][3].refutation_result['p_value'] if x[1] else None for x in results_full]
df_res['rand_dml-is_statistically_significant'] = [x[1][3].refutation_result['is_statistically_significant'] if x[1] else None for x in results_full]
df_res['unobserved_dml'] = [x[1][4].new_effect if x[1] else None for x in results_full]
df_res['placebo_dml'] = [x[1][5].new_effect if x[1] else None for x in results_full]
df_res['dml_pl_p_val'] = [x[1][5].refutation_result['p_value'] if x[1] else None for x in results_full]
df_res['dml_pl_is_statistically_significant'] = [x[1][5].refutation_result['is_statistically_significant'] if x[1] else None for x in results_full]

In [ ]:
df_res['modifiers'] = [x[2] for x in results_full]
df_res['confounders'] = [x[3] for x in results_full]

In [ ]:
df_res['backdoor_var'] = [x[4] for x in results_full]

In [ ]:
df_res